# Object Tracking using KCF Tracker
**Full Coding available at [object_tracking.py](object_tracking.py)** 
<br>
This is the documentation for Object Tracking using OpenCV (KCF Tracker) 

***Libraries Used In The Script***
- Motor_Encoder
- PCA9685_MC 
- OpenCV 
- Picamera2 
- Libcamera 
- Numpy



## Installing Libraries @ Dependencies 
- Build-In Libraries 
    - Picamera2 
    - libcamera 
    - PCA9685_MC 
    - Motor Encoder 

- User Installed Libraries
    - OpenCV 
    - numpy 

### First, let's install the user install libraries ! 
1. OpenCV 
    - Follow the link [OpenCV.org](https://docs.opencv.org/4.x/d2/de6/tutorial_py_setup_in_ubuntu.html)
2. NumPy 
    - Follow the link [numpy.org](https://numpy.org/install/)


### Verify the installed libraries 
1. OpenCV 
    - In Python File type: <br>
    `import cv2` <br>
    `print(cv2.__version__)`
    - The coding above will print the version of opencv library installed
      
2. NumPy 
    - In Python File type: <br>
    `import numpy as np`
    - Installation sucess if coding run without error 

## Let's Start Coding ! 
### 1. Import all the libraries 
- OpenCV 
- NumPy 
- Picamera2 
- libcamera
- Motor_Encoder 
- PCA9685_MC
- time

In [ ]:
import cv2
from picamera2 import Picamera2
from libcamera import controls
from PCA9685_MC import Motor_Controller
from Motor_Encoder import Encoder
import numpy as np

### 2. Initialise all the libraries in a function 
- Setup `KCF` Tracker 
    ```py
    tracker = cv2.TrackerKCF_create() 
    ``` 
- Set up the camera with a specific heigth and width 

    ```py
    frame_width = 640  
    frame_height = 480
    cap = Picamera2()
    cap.configure(cap.create_preview_configuration(main={"format": 'XRGB8888', "size": (frame_height, frame_width)}))
    cap.set_controls({"AfMode": controls.AfModeEnum.Continuous})
    cap.start()
    ```
- Set up Motor and Encoder 
    ```py 
    motor = Motor_Controller()
    enc = Encoder()
    ```

In [ ]:
tracker = cv2.TrackerKCF_create() 
# cap = cv2.VideoCapture(0)
# frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = 640  
frame_height = 480
cap = Picamera2()
cap.configure(cap.create_preview_configuration(main={"format": 'XRGB8888', "size": (frame_height, frame_width)}))
cap.set_controls({"AfMode": controls.AfModeEnum.Continuous})
cap.start()

motor = Motor_Controller()
enc = Encoder()

### 3. The `tracking` function 
- This function is responsible for tracking an object after the bounding box is set.
- It displays the current coordinates of the tracked object and controls motor movement based on the object's position within the frame.
- A red dot is drawn at the center of the tracked object for visual reference.

- **Draw Bounding Box and Coordinates**
    - The bounding box is drawn around the tracked object using a green rectangle.
    - The coordinates (X, Y) of the top-left corner of the bounding box are displayed on the frame.

        ```py
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"X:{x} , Y: {y}", (x ,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        ```

- **Draw Center Dot**
    - The center of the bounding box is calculated, and a red dot is drawn at the center of the tracked object.

        ```py
        center_x = x + w // 2
        center_y = y + h // 2
        cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)  # Red dot at center
        ```

- **Motor Movement Based on Object Position**
    - The motor's movement is controlled by checking the center of the bounding box. If the object is in different parts of the frame, it turns left, right, or moves forward. Slower speeds are used when the object is closer to the center of the frame.
    
        ```py
        if center_y < 220 and center_y > 100:
            if center_x < 300:
                motor.AntiClock_Rotate(20)  # Turn left
            if center_x > 340:
                motor.Clock_Rotate(20)      # Turn right
            else:
                motor.Forward(20)           # Move forward
        elif center_y > 240 and center_y < 440:
            if center_x < 300:
                motor.AntiClock_Rotate(10)  # Slow left
            if center_x > 340:
                motor.Clock_Rotate(10)      # Slow right
            else:
                motor.Forward(10)           # Slow forward
        ```

- **Stop Movement**
    - If the object is not within the defined regions, the motor will stop.
    
        ```py
        else:
            motor.Brake()  # Stop motor
        ```

In [ ]:
def tracking(frame, x,y,w,h):
    
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(frame, "Tracking", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(frame, "Press 'q' to quit", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(frame, f"X:{x} , Y: {y}", (x ,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    # Calculate the center of Bounding Box 
    center_x = x + w // 2
    center_y = y + h // 2
    cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)  # Draw a circle at the center
    # Tracking Logic 
    enc.encoder() 
    if center_y < 220 and center_y > 100:
        if center_x < 300:
            print("Turn Left")
            motor.AntiClock_Rotate(20) 
        if center_x > 340:
            print("Turn Right")
            motor.Clock_Rotate(20)
        else:
            motor.Forward(20)
    # Slow Approch        
    elif center_y > 240 and center_y < 440:
        if center_x < 300:
            print("Turn Left")
            motor.AntiClock_Rotate(10)
        if center_x > 340:
            print("Turn Right")
            motor.Clock_Rotate(10)
        else:
            motor.Forward(10)
    else:
        motor.Brake()
    

### 4. The `main` function

- This function initializes the tracker and handles the object tracking process in a loop.
- It captures the video frames from the camera, updates the tracker, and controls the motor based on the tracked object's position.

- **Capture the First Frame and Select ROI**
    - The camera captures the initial frame, and the user is prompted to select the **Region of Interest (ROI)** for tracking. The selected ROI is then passed to the tracker for initialization.

        ```py
        frame = cap.capture_array()
        bbox = cv2.selectROI(frame, showCrosshair=True, fromCenter=False)
        cv2.destroyWindow("ROI selector")

        tracker.init(frame, bbox)
        ```

- **Tracking Loop**
    - The video stream is continuously processed inside a loop.
    - For each frame, the tracker is updated, and the bounding box is redrawn.
    - If tracking is successful, the `tracking()` function is called to update the motor’s movement based on the object’s location.

        ```py
        while True:
            frame = cap.capture_array()  # Read frame
            success, box = tracker.update(frame)  # Update the tracker with the new frame
            if success:
                (x, y, w, h) = [int(v) for v in box]
                tracking(frame, x,y,w,h)  # Call the tracking function
        ```

- **Handle Tracking Failure**
    - If the tracker fails to follow the object, a **Tracking failure** message is displayed on the frame, and the motor stops (brakes).

        ```py
        else:
            cv2.putText(frame, "Tracking failure", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            motor.Brake()  # Stop motor
        ```

- **Display Frame and Exit Condition**
    - The tracked frame is displayed with the bounding box and other details.
    - The user can press the **'q'** key to exit the tracking loop.

        ```py
        cv2.imshow('Tracking_Area', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        ```

- **Stop the Camera**
    - Once the loop exits, the camera is stopped.

        ```py
        cap.stop()
        ```


In [ ]:
def main(): 
    frame = cap.capture_array()
    bbox = cv2.selectROI(frame, showCrosshair=True, fromCenter=False)
    cv2.destroyWindow("ROI selector")
 
    # Initialize tracker with first frame and bounding box
    tracker.init(frame, bbox)

    # if not cap.isOpened():
    #     print("Error: Could not open video capture.")
    #     return
    # print(f"Tracking started with ROI: {bbox}")
    
    while True:
        frame = cap.capture_array()  # Read frame
        # Tracking mode: update the tracker and draw the tracked bounding box
        success, box = tracker.update(frame)  # Update the tracker with the new frame
        if success:
            (x, y, w, h) = [int(v) for v in box]
            tracking(frame, x,y,w,h)
        else:
            cv2.putText(frame, "Tracking failure", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            motor.Brake() 
        cv2.imshow('Tracking_Area', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.stop()

### 4. Wraping up 
- when the programe start run the main function 
    ```py
    try:
        if __name__ == '__main__': 
            main()
    ``` 
- When there is keyboard interupt (Ctrl + c)
    ```py
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
    ```
- When the programe ends 
    ```py 
    finally:
        motor.cleanup()
        enc.stop()
        cap.stop()
        cv2.destroyAllWindows()
        print("Program Terminated \nExiting....")
    ``` 

In [ ]:

try:
    if __name__ == '__main__':
        main()
except KeyboardInterrupt:
    print("KeyboardInterrupt")
    
finally:
    motor.cleanup()
    enc.stop()
    cap.stop()
    cv2.destroyAllWindows()
    print("Program Terminated \nExiting....")